<a href="https://colab.research.google.com/github/ccstevie/nhl_model/blob/main/NHL_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from datetime import date, timedelta
from functools import reduce

In [ ]:
# we will use the last 30 days as our data range
today = date.today()
print("Today's date:", today)
start = today - timedelta(days=30)
print("30 days ago:", start)

Today's date: 2024-02-21
30 days ago: 2024-01-22


In [ ]:
url = f"https://www.naturalstattrick.com/teamtable.php?fromseason=20232024&thruseason=20232024&stype=2&sit=5v5&score=all&rate=n&team=all&loc=B&gpf=410&fd={start}&td={today}"
req = requests.get(url)
req.status_code

200

In [ ]:
df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
df.head()

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,...,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO
1,Winnipeg Jets,9,438:19,3,5,1,3,7,0.389,464,...,85,55.96,1,1,50.0,0.93,98.82,4.57,92.63,0.972
2,Boston Bruins,11,518:55,5,4,2,4,12,0.545,447,...,125,37.19,2,3,40.0,2.70,97.60,8.89,94.36,1.032
3,Vegas Golden Knights,9,452:19,5,3,1,5,11,0.611,386,...,106,41.76,2,2,50.0,2.63,98.11,11.00,92.92,1.039
4,New Jersey Devils,11,520:21,5,5,1,5,11,0.500,548,...,124,52.67,1,1,50.0,0.72,99.19,8.62,91.55,1.002
5,Florida Panthers,11,523:18,10,1,0,9,20,0.909,590,...,108,56.63,4,1,80.0,2.84,99.07,8.10,95.42,1.035


In [ ]:
# sort by CF% GF% XGF% HDCF% SH% and obtain rankings for each team
cf = df.sort_values(by="CF%", ascending=False, ignore_index=True)
cf.index += 1
cf.reset_index(inplace=True)
cf = cf.rename(columns = {"index":"CF%"})
cf = cf[["Team", "CF%"]]
# print cf as an example
cf

,Team,CF%,CF%
0,Carolina Hurricanes,1,60.58
1,Los Angeles Kings,2,54.90
2,Vancouver Canucks,3,54.76
3,Winnipeg Jets,4,54.65
4,Florida Panthers,5,54.38
5,Philadelphia Flyers,6,53.97
6,Edmonton Oilers,7,53.25
7,Calgary Flames,8,53.17
8,Seattle Kraken,9,53.06
9,Tampa Bay Lightning,10,52.62


In [ ]:
gf = df.sort_values(by="GF%", ascending=False, ignore_index=True)
gf.index += 1
gf.reset_index(inplace=True)
gf = gf.rename(columns = {"index":"GF%"})
gf = gf[["Team", "GF%"]]

In [ ]:
xgf = df.sort_values(by="xGF%", ascending=False, ignore_index=True)
xgf.index += 1
xgf.reset_index(inplace=True)
xgf = xgf.rename(columns = {"index":"xGF%"})
xgf = xgf[["Team", "xGF%"]]

In [ ]:
hdcf = df.sort_values(by="HDCF%", ascending=False, ignore_index=True)
hdcf.index += 1
hdcf.reset_index(inplace=True)
hdcf = hdcf.rename(columns = {"index":"HDCF%"})
hdcf = hdcf[["Team", "HDCF%"]]

In [ ]:
sh = df.sort_values(by="SH%", ascending=False, ignore_index=True)
sh.index += 1
sh.reset_index(inplace=True)
sh = sh.rename(columns = {"index":"SH%"})
sh = sh[["Team", "SH%"]]

In [ ]:
#merge all DataFrames into one
dfs = [cf, gf, xgf, hdcf, sh]
final_df = reduce(lambda  left,right: pd.merge(left,right,on=['Team'],
                                            how='outer'), dfs)
final_df

,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
0,Carolina Hurricanes,1,60.58,2,63.33,2,57.29,5,56.61,23,7.63
1,Los Angeles Kings,2,54.90,30,38.46,7,54.39,18,51.41,30,5.86
2,Vancouver Canucks,3,54.76,6,58.14,3,56.30,4,56.63,2,11.06
3,Winnipeg Jets,4,54.65,28,41.67,10,53.69,10,53.67,32,4.57
4,Florida Panthers,5,54.38,1,67.65,4,55.63,13,52.97,18,8.10
5,Philadelphia Flyers,6,53.97,31,37.84,13,52.91,3,56.82,29,6.19
6,Edmonton Oilers,7,53.25,4,60.53,8,54.33,2,58.13,8,9.66
7,Calgary Flames,8,53.17,14,51.35,5,54.96,19,50.90,22,7.69
8,Seattle Kraken,9,53.06,10,57.14,9,54.16,8,54.82,27,7.08
9,Tampa Bay Lightning,10,52.62,23,45.83,17,50.50,21,49.71,6,10.09


In [ ]:
# ex. get a team's stats
final_df.loc[final_df["Team"] == "Toronto Maple Leafs"]

,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
20,Toronto Maple Leafs,21,48.56,11,56.67,14,51.06,6,55.56,20,8.06
